# Assignment 19- Balancing Data with Repetition

## Upload and Clean Data

In [1]:
loans <- read.csv("C:/Users/student/Documents/Data Analysis Practice Data Sets/Default_Loan_Data.csv", 
                  na.strings = c(NA, ""))
str(loans)
loans$BAD <- as.factor(loans$BAD)

library(Quick.Analysis)
sum(is.na(loans))
loans.clean<- quick_clean(loans, "impute")
sum(is.na(loans.clean))

'data.frame':	5960 obs. of  13 variables:
 $ BAD    : int  1 1 1 1 0 1 1 1 1 1 ...
 $ LOAN   : int  1100 1300 1500 1500 1700 1700 1800 1800 2000 2000 ...
 $ MORTDUE: num  25860 70053 13500 NA 97800 ...
 $ VALUE  : num  39025 68400 16700 NA 112000 ...
 $ REASON : Factor w/ 2 levels "DebtCon","HomeImp": 2 2 2 NA 2 2 2 2 2 2 ...
 $ JOB    : Factor w/ 6 levels "Mgr","Office",..: 3 3 3 NA 2 3 3 3 3 5 ...
 $ YOJ    : num  10.5 7 4 NA 3 9 5 11 3 16 ...
 $ DEROG  : int  0 0 0 NA 0 0 3 0 0 0 ...
 $ DELINQ : int  0 2 0 NA 0 0 2 0 2 0 ...
 $ CLAGE  : num  94.4 121.8 149.5 NA 93.3 ...
 $ NINQ   : int  1 0 1 NA 0 1 1 0 1 0 ...
 $ CLNO   : int  9 14 10 NA 14 8 17 8 12 13 ...
 $ DEBTINC: num  NA NA NA NA NA ...


[1] 5271

[1] 0

## 1. Train a random forest using undersampling. Let rf1 to be the trained model

In [2]:
library(caret)
library(ranger)

split <- createDataPartition(loans.clean$BAD, p = .7, list=FALSE, times=1)
training <- loans.clean[split,]
testing <- loans.clean[-split,]

train1 <- training[training$BAD == "1",]
train0 <- training[training$BAD == "0",]

n1 <- nrow(train1)
n0 <- nrow(train0)

train00 <- train0[sample(1:n0, n1),]
train_under <- rbind(train00, train1)

rf1 <- ranger(BAD~., data=train_under)

Loading required package: lattice
Loading required package: ggplot2



## 2. Use rf1 to produce the prediction for the test set. 

In [4]:
n=1000
m = matrix(0, nrow = nrow(testing), ncol = n)
m[,1] <- predict(rf1, data = testing, positive = "1")$predictions
m[1:5,1]

[1] 2 2 2 2 2

## 3. Redo 1 and 2 to create c2, c3, c4,…, c1000

In [5]:
library(ranger)
n=1000

for (i in 2:n){
  train00 <- train0[sample(1:n0, n1),]
  train_under <- rbind(train00, train1)
  tree_under <- ranger(BAD~., data=train_under)
  m[,i] <- predict(tree_under, data = testing, positive = "1")$prediction
}

m <- m - 1

## 5.	Let cc be a row mode of mc.  The vector cc is the majority vote of 1000 predictions for the test set. 

In [6]:
most_frequent <- function(x){
   unique_x <- unique(x)
   unique_x[which.max(tabulate(match(x, unique_x)))]
}

cc <- apply(m, 1, most_frequent)
cc <- as.factor(cc)

## 6.	Produce the confusion matrix based on cc and the true target.

In [7]:
confusionMatrix(cc, reference = testing$BAD, positive = "1")

Confusion Matrix and Statistics

          Reference
Prediction    0    1
         0 1293   39
         1  138  317
                                          
               Accuracy : 0.901           
                 95% CI : (0.8862, 0.9144)
    No Information Rate : 0.8008          
    P-Value [Acc > NIR] : < 2.2e-16       
                                          
                  Kappa : 0.7189          
 Mcnemar's Test P-Value : 1.757e-13       
                                          
            Sensitivity : 0.8904          
            Specificity : 0.9036          
         Pos Pred Value : 0.6967          
         Neg Pred Value : 0.9707          
             Prevalence : 0.1992          
         Detection Rate : 0.1774          
   Detection Prevalence : 0.2546          
      Balanced Accuracy : 0.8970          
                                          
       'Positive' Class : 1               
                                          